In [2]:
import numpy as np

from utils.inference import ecg_noise_quantification

checkpoint_path ="./output/final_model.pth"

# fake ecg
ecg = np.random.randn(12,7000)

1


In [3]:
output =ecg_noise_quantification(
    ecg=ecg,
    sampling_freq=500,
    checkpoint_path=checkpoint_path,
)


2


3
